In [1]:
import os
import torchvision
import torch
from ultralytics import YOLO
import cv2
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
os.environ["YOLOv8_DDP_DISABLE"] = "True"
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
# Define paths
root_path = 'C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data'
#model_path = 'C:/Users/Priyam/OneDrive/Documents/Capstone gpt/runs/detect/train27/weights/best.pt'
model_path = os.path.join(root_path, 'yolov8l.pt')
image_path = os.path.join(root_path, 'images')
label_path = os.path.join(root_path, 'labels')
video_path = 'C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data/test_litter_small.mp4'
output_video_path = 'C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data/output_test_litter_small_annotated.mp4'
yaml_path = os.path.join(root_path, 'litter_dataset.yaml')

In [2]:
torch.cuda.empty_cache() 

In [3]:
yaml_content = '''
path: C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data/
train: images/train
val: images/val
test: images/test
names:
  0: litter
'''

with open(yaml_path, 'w') as file:
    file.write(yaml_content)

In [4]:

# Set device to GPU (CUDA) if available, otherwise fallback to CPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
model = YOLO(model_path).to(device)


Using device: cuda


In [ ]:

def train_yolo_with_early_stopping(model, yaml_path, imgsz, epochs, patience):
    best_mAP50, wait = 0, 0
    mAP50_values = []

    # Setup plot for live training metrics
    plt.ion()
    fig, ax = plt.subplots()
    ax.set_xlabel('Epoch')
    ax.set_ylabel('mAP50')
    ax.set_title('Training Metrics - Early Stopping & Overfitting Detection')

    train_params = {
        'epochs': epochs,       # Set to the passed epochs parameter
        'lr0': 0.001,           # Initial learning rate
        'lrf': 0.2,             # Final learning rate fraction
        'optimizer': 'SGD',     # Optimizer choice
        'cos_lr': True,         # Cosine annealing scheduler   
        'batch': 16,       # Adjust batch size based on GPU memory
        'half': True,            # Use mixed precision for efficiency
        'augment': True
    }

    # Run training with current parameters
    results = model.train(data=yaml_path, imgsz=640, **train_params,patience=10)
    # Extract mAP50 and track it
    mAP50 = results.box.map50
    mAP50_values.append(mAP50)
    ax.plot(range(len(mAP50_values)), mAP50_values, label="mAP50", color='blue')
    ax.legend()
    # Display live plot updates
    clear_output(wait=True)
    plt.pause(0.1)
    # Early stopping logic based on mAP50 improvement
    if mAP50 > best_mAP50:
        best_mAP50, wait = mAP50, 0  # Reset patience counter if mAP50 improves
    else:
        wait += 1
        if wait >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1} with best mAP50: {best_mAP50:.3f}")

    plt.ioff()  # Turn off interactive mode
    return results

In [6]:
results_640 = train_yolo_with_early_stopping(model, yaml_path, imgsz=640, epochs=50, patience=10)

engine\trainer: task=detect, mode=train, model=C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data\yolov8l.pt, data=C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data\litter_dataset.yaml, epochs=50, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train60, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

train: Scanning C:\Users\Priyam\OneDrive\Documents\Capstone gpt\data\labels\train.cache... 359 images, 1 backgrounds, 0 corrupt: 100%|██████████| 360/360 [00:00<?, ?it/s]
val: Scanning C:\Users\Priyam\OneDrive\Documents\Capstone gpt\data\labels\val.cache... 43 images, 2 backgrounds, 0 corrupt: 100%|██████████| 45/45 [00:00<?, ?it/s]


Plotting labels to runs\detect\train60\labels.jpg... 
optimizer: SGD(lr=0.001, momentum=0.937) with parameter groups 103 weight(decay=0.0), 110 weight(decay=0.0005), 109 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train60
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      10.7G      2.781      3.925       1.79         72        640: 100%|██████████| 23/23 [03:06<00:00,  8.09s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:13<00:00,  6.78s/it]

                   all         45        140    0.00526      0.507    0.00577    0.00158


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50      10.7G      1.652      0.955      1.068         90        640: 100%|██████████| 23/23 [02:12<00:00,  5.75s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:10<00:00,  4.96s/it]
                   all         45        140      0.701      0.479      0.695      0.332



50 epochs completed in 0.971 hours.
Optimizer stripped from runs\detect\train60\weights\last.pt, 79.3MB
Optimizer stripped from runs\detect\train60\weights\best.pt, 79.3MB

Validating runs\detect\train60\weights\best.pt...
Ultralytics 8.3.5  Python-3.12.4 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 286 layers, 39,433,555 parameters, 0 gradients, 145.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:04<00:00,  2.44s/it]


                 all         45        140      0.714      0.493     0.714      0.34
Speed: 0.8ms preprocess, 93.0ms inference, 0.0ms loss, 7.2ms postprocess per image
Results saved to runs\detect\train60


In [7]:
import tensorrt as trt

# TensorRT logger
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

def build_engine(onnx_file_path, engine_file_path):
    with trt.Builder(TRT_LOGGER) as builder, \
         builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)) as network, \
         trt.OnnxParser(network, TRT_LOGGER) as parser:
        
        # Load the ONNX model
        with open(onnx_file_path, 'rb') as model:
            if not parser.parse(model.read()):
                print('Failed to parse ONNX model')
                for error in range(parser.num_errors):
                    print(parser.get_error(error))
                return None

        # Configure the builder
        config = builder.create_builder_config()
        
        # Set memory pool limit (equivalent to max_workspace_size)
        config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30)  # 1 GiB

        # Enable FP16 precision if supported
        if builder.platform_has_fast_fp16:
            config.set_flag(trt.BuilderFlag.FP16)

        # Create an optimization profile
        profile = builder.create_optimization_profile()
        input_shape = network.get_input(0).shape  # Assume the first input is the batch dimension
        profile.set_shape(network.get_input(0).name, min=input_shape, opt=input_shape, max=input_shape)
        config.add_optimization_profile(profile)

        # Build serialized engine
        serialized_engine = builder.build_serialized_network(network, config)
        if serialized_engine is None:
            print("Failed to build the engine")
            return None

        # Deserialize engine
        runtime = trt.Runtime(TRT_LOGGER)
        engine = runtime.deserialize_cuda_engine(serialized_engine)

        # Save the engine
        with open(engine_file_path, "wb") as f:
            f.write(engine.serialize())
        print(f"TensorRT engine created and saved to {engine_file_path}")

# Define the paths for the ONNX and engine files
onnx_path=os.path.join(root_path, 'yolov8l.onnx') # Path to the ONNX file
engine_file = 'yolov8l_fp16.engine'  # Path to save the TensorRT engine
build_engine(onnx_path, engine_file)

TensorRT engine created and saved to yolov8l_fp16.engine


In [12]:
engine_path='C:/Users/Priyam/OneDrive/Documents/Capstone gpt/yolov8l_fp16.engine'
video_path='C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data/test_litter_small.mp4'
output_video_path='C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data/output_test_litter_small.mp4'

In [13]:
import pycuda.driver as cuda
import pycuda.autoinit
import cv2
import numpy as np
import time

In [14]:
input_shape = (1, 3, 640, 640)  # Adjust to match the model input size
output_shape = (1, 25200, 85)   # Adjust based on the YOLO model output
d_input = cuda.mem_alloc(int(np.prod(input_shape) * np.float32().nbytes))
d_output = cuda.mem_alloc(int(np.prod(output_shape) * np.float32().nbytes))

In [15]:
# Path to the engine file and video input
engine_path='C:/Users/Priyam/OneDrive/Documents/Capstone gpt/yolov8l_fp16.engine'
video_path='C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data/test_litter_small.mp4'
output_video_path='C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data/output_test_litter_small.mp4'

# Set up the TensorRT logger and load the engine
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

def load_engine(engine_path):
    with open(engine_path, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
        return runtime.deserialize_cuda_engine(f.read())

engine = load_engine(engine_path)
context = engine.create_execution_context()

# Define input and output shapes based on the model's expected format
input_shape = (1, 3, 640, 640)  # Adjust to match the model input size
output_shape = (1, 25200, 85)   # Adjust based on the YOLO model output

# Allocate device memory for input and output
#d_input = cuda.mem_alloc(1 * np.prod(input_shape) * np.float32().nbytes)
#d_output = cuda.mem_alloc(1 * np.prod(output_shape) * np.float32().nbytes)
bindings = [int(d_input), int(d_output)]

# Open the video file
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Prepare the output video writer
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Helper function to preprocess each frame
def preprocess_frame(frame, input_shape):
    frame_resized = cv2.resize(frame, (input_shape[3], input_shape[2]))  # Resize to model input size
    frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
    frame_normalized = frame_rgb / 255.0  # Normalize to [0, 1]
    frame_transposed = np.transpose(frame_normalized, (2, 0, 1)).astype(np.float32)  # Change to (C, H, W)
    return np.expand_dims(frame_transposed, axis=0)

# Function to post-process the output (customized for YOLOv8 output format)
def postprocess_output(output, conf_threshold=0.1, width=640, height=640):
    detections = []
    # Iterate over each detection
    for detection in output[0]:
        confidence = detection[4]
        if confidence > conf_threshold:
            x_center, y_center, w, h = detection[0:4]
            class_id = np.argmax(detection[5:])
            x = int((x_center - w / 2) * width)  # Convert from center x, y, w, h to box coordinates
            y = int((y_center - h / 2) * height)
            w = int(w * width)
            h = int(h * height)
            detections.append((x, y, w, h, confidence, class_id))
    return detections

# Inference loop over the video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Start timer to calculate FPS
    start_time = time.time()
    
    # Preprocess the frame
    input_image = preprocess_frame(frame, input_shape)
    input_image = np.ascontiguousarray(input_image)
    
    # Transfer the input data to the GPU
    cuda.memcpy_htod(d_input, input_image)

    # Run inference
    context.execute_v2(bindings=bindings)

    # Transfer predictions back from the GPU
    output = np.empty(output_shape, dtype=np.float32)
    cuda.memcpy_dtoh(output, d_output)

    # Post-process the output
    detections = postprocess_output(output, conf_threshold=0.1, width=width, height=height)

    # Draw detections on the frame
    for (x, y, w, h, confidence, class_id) in detections:
        # Draw bounding box and label
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f'Class {class_id} {confidence:.2f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Calculate and display FPS
    fps = 1.0 / (time.time() - start_time)
    cv2.putText(frame, f'FPS: {fps:.2f}', (width - 120, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Write the frame with detections to the output video
    out.write(frame)

In [16]:
# Release resources
cap.release()
out.release()
print("Inference complete. Output video saved to:", output_video_path)

Inference complete. Output video saved to: C:/Users/Priyam/OneDrive/Documents/Capstone gpt/data/output_test_litter_small.mp4


In [18]:
print(fps)

74
